In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

from sklearn.utils import shuffle


# 데이터 다운로드 
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

C:\Users\jeeyeon\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
train_xdata = np.array([np.reshape(x, (28,28,1)) for x in mnist.train.images])  # (H, W, C)
test_xdata = np.array([np.reshape(x, (28,28,1)) for x in mnist.test.images])    # (H, W, C)

train_labels = mnist.train.labels
test_labels = mnist.test.labels

In [4]:
train_xdata.shape, train_labels.shape, test_xdata.shape, test_labels.shape

((55000, 28, 28, 1), (55000, 10), (10000, 28, 28, 1), (10000, 10))

In [5]:

I_D = tf.placeholder(tf.float32,[None ,28,28,1])   # [N, W, H, C]
Y = tf.placeholder(tf.float32, [None, 10])         # [N, output class]

C_W1 = tf.Variable(tf.truncated_normal([4, 4, 1, 32]))  # [FW1, FH1, C1, FN1]
C_W2 = tf.Variable(tf.truncated_normal([4, 4, 32,64])) # [FW2, FH2, C2, FN2]



In [6]:
#Convolution1
conv1 = tf.nn.conv2d(I_D, C_W1, strides = [1,1,1,1], padding = "SAME")   # stride = [1,*,*,1] 에서 * : 이동할 filter step의미함
bias1 = tf.Variable(tf.zeros([1,1,1,32]))                                # FN : filter 수만큼 bias 생성해줌 
relu1 = tf.nn.relu( conv1 + bias1 )                                      # relu ( conv1 + bias1 ) 
max_pool1 = tf.nn.max_pool(relu1, ksize = [1,2,2,1], strides = [1,2,2,1], padding ="SAME")  # ksize  : filtersize

#Convolution2
conv2 = tf.nn.conv2d(max_pool1, C_W2, strides = [1,1,1,1], padding = "SAME")
bias2 = tf.Variable(tf.zeros([1,1,1,64]))
relu2 = tf.nn.relu(conv2 + bias2)
max_pool2 = tf.nn.max_pool(relu2, ksize = [1,2,2,1], strides = [1,2,2,1], padding ="SAME") 

max_pool1.shape, max_pool2.shape

(TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(32)]),
 TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(64)]))

In [7]:
final_conv_shape = max_pool2.shape
final_shape = int(final_conv_shape[1]* final_conv_shape[2] * final_conv_shape[3])  # int 안해주면 인식안돼 ㅠㅠ

flat_output = tf.reshape(max_pool2, [-1, final_shape])
# flat_output = tf.reshape(max_pool2, [None, final_shape])  # 왜 None 으로 하면 안돼 ? ㅠㅠ 

print(final_shape)
print(flat_output.shape)

3136
(?, 3136)


In [8]:
n_hidden = 128


# Affine layer 1
# 만들고자 하는 형태는 z1 = x * W1 + b1 -> return : z1=relu(z1)
W1 = tf.Variable(tf.truncated_normal([final_shape, n_hidden], mean = 0, stddev = tf.sqrt(2/final_shape) )) #variable이니까 initializing 해야한다.
b1 = tf.Variable(tf.zeros([1, n_hidden]))  # 1부분이 broad casting 됨
z1 = tf.nn.relu(tf.matmul(flat_output,W1) + b1)


# Affine layer 2
W2 = tf.Variable(tf.truncated_normal([n_hidden, 10], stddev = tf.sqrt(2/n_hidden)))  
b2 = tf.Variable(tf.zeros([1, 10]))    
z2 = tf.nn.softmax(tf.matmul(z1,W2) + b2)

y = z2



In [9]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(y), axis = 1))
train_step = tf.train.AdagradOptimizer(0.05).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



In [ ]:
N = 55000
batch_size = 128 
n_batches = N // batch_size   # 몇번 할건지!!!!! 

# 작성하기 전에 작이제까지 작성한 변수들을 초기화해야함!
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)


for epoch in range(10): 
    X_, Y_ = shuffle(train_xdata, train_labels)
    
    #j = sess.run(accuracy, feed_dict = { I_D: train_xdata, Y: train_labels })
    #print(epoch, j)
    
    for i in range(n_batches):  # 한 데이터당 15번해줘야하니까 
        start = i * batch_size
        end = start + batch_size  # end가 300번보다 많으면 error발생함  # 이렇게해도 300넘지않음
        
        sess.run(train_step, feed_dict = {I_D : X_[start:end, :], Y : Y_[start:end, :]})
    

In [ ]:
# 모델 평가하기

my_prediction = tf.cast( tf.argmax(y,1), tf.float32 )    

correct_prediction = tf.equal( tf.argmax(y,1), tf.argmax(Y,1) )  # argmax 몇번째인지 찾아줌
    # tf.argmax(y,1) : 우리 모델이 생각하기에 가장 적합하다고 판단한 라벨
    # tf.argmax(Y,1) : 실제라벨

    # tf.equal 을 통해 우리 예측이 맞았는지 확인가능
    # 부울값으로 이루어진 리스트 얻게 됨 

    # 부동소수점으로 변환후 평균 계산함
    # ex. [T,F,T,T] 는 [1,0,1,1] 이 되며 이 평균을 계산하면 0.75가 됨!
    
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))   

    # correct_prediction 값을 float32 형식으로 변환함

print(sess.run(accuracy, feed_dict={x: mnist.test.images, Y: mnist.test.labels}))
